In [1]:

!pip install opendatasets 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=a78d48b90bcc80b3822128bd7107a64f5bb6c1c1c3642cd7672cd4fb2a3289ac
  Stored in directory: /root/.cache/pip/wheels/ac/b2/c3/fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


In [2]:
from keras.models import Sequential
from keras.optimizers import Adam
import glob
from PIL import Image
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import keras


In [3]:
import opendatasets as od

od.download("https://www.kaggle.com/competitions/generative-dog-images")

Skipping, found downloaded files in "./generative-dog-images" (use force=True to force download)


In [4]:
!unzip generative-dog-images/all-dogs.zip

Archive:  generative-dog-images/all-dogs.zip
replace all-dogs/n02109961_16718.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
dataset = keras.utils.image_dataset_from_directory(
    "all-dogs", label_mode=None, image_size=(64, 64), batch_size=16
)
dataset = dataset.map(lambda x: x / 255.0)

Found 20579 files belonging to 1 classes.


In [9]:
tamanho_do_ruido = 128


Descriminador = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="Descriminador",
)
Descriminador.summary()





Gerador = keras.Sequential(
    [
        keras.Input(shape=(tamanho_do_ruido,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="Gerador",
)
print(Gerador.summary())
















SyntaxError: invalid syntax (656817868.py, line 44)

In [12]:

class GAN(keras.Model):
    def __init__(self, Descriminador, Gerador, tamanho_do_ruido=128):
        super(GAN, self).__init__()#roda o init d keras.model
        self.Descriminador = Descriminador
        self.Gerador = Gerador
        self.tamanho_do_ruido = tamanho_do_ruido

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        
        batch_size = tf.shape(real_images)[0]
        #                     quantas imgs tem na base d dados
        ruido = tf.random.normal(shape=(batch_size, self.tamanho_do_ruido))


        imgs_geradas = self.Gerador(ruido)


        imgs_combinadas = tf.concat([imgs_geradas, real_images], axis=0)

        #imgs falsas = 1    imgs reais = 0
        classes = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        
        #por algum motivo do alem se vc add ruido na classe os resultados ficam melhores então...
        classes += 0.05 * tf.random.uniform(tf.shape(classes))

        # treina o descriminador
        with tf.GradientTape() as tape:
            predictions = self.Descriminador(imgs_combinadas)
            d_loss = self.loss_fn(classes, predictions)
        grads = tape.gradient(d_loss, self.Descriminador.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.Descriminador.trainable_weights)
        )

        # mais ruido
        mais_ruido = tf.random.normal(shape=(batch_size, self.tamanho_do_ruido))

        # Assemble classes that say "all real images"
        misleading_classes = tf.zeros((batch_size, 1))

        # Train the Gerador (note that we should *not* update the weights
        # of the Descriminador)!
        with tf.GradientTape() as tape:
            predictions = self.Descriminador(self.Gerador(mais_ruido))
            g_loss = self.loss_fn(misleading_classes, predictions)
        grads = tape.gradient(g_loss, self.Gerador.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.Gerador.trainable_weights))

        # atualiza as metricas
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [13]:
!mkdir generated_img_
class Callback_q_monitora_a_gan(keras.callbacks.Callback):
    def __init__(self, num_img=2, tamanho_do_ruido=128):
        self.num_img = num_img
        self.tamanho_do_ruido = tamanho_do_ruido

    def on_epoch_end(self, epoch, logs=None):
        # with open('Gerador.pickle', 'wb') as f:
        #     pickle.dump(self.model.Gerador, f)
        # with open('Descriminador.pickle', 'rb') as f:
        #     self.model.Descriminador = pickle.load(f)
            
        #salva com keras
        self.model.Descriminador.save('Descriminador_keras.h5')
        self.model.Gerador.save('Gerador_keras.h5')
        
        ruido = tf.random.normal(shape=(self.num_img, self.tamanho_do_ruido))
        imgs_geradas = self.model.Gerador(ruido)
        imgs_geradas = 0.5 * imgs_geradas + 0.5
        imgs_geradas.numpy()
        for i in range(self.num_img):
            img = keras.utils.array_to_img(imgs_geradas[i])
            img.save("generated_img_%03d_%d.png" % (epoch, i))

mkdir: cannot create directory ‘generated_img_’: File exists


In [14]:

epochs = 100 

gan = GAN(Descriminador=Descriminador, Gerador=Gerador, tamanho_do_ruido=tamanho_do_ruido)
gan.compile(
    d_optimizer=keras.optimizers.adam_v2.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.adam_v2.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[Callback_q_monitora_a_gan(num_img=10, tamanho_do_ruido=tamanho_do_ruido)]
)

#salva a classe 
with open('gan.pickle', 'wb') as f:
    pickle.dump(gan, f)


Epoch 1/100
1287/1287 [==============================] - 1189s 917ms/step - d_loss: 15.4769 - g_loss: 2122.5100
Epoch 2/100
 618/1287 [=============>................] - ETA: 10:09 - d_loss: 13.1356 - g_loss: 1044.3911

KeyboardInterrupt: 